In [1]:
import main_rnn, main_cnn
from icecream import ic
import numpy as np

In [2]:
from src.layer.dense import Dense
from src.layer.lstm import LSTM
from src.sequential import Sequential
from src.layer.detector import Detector
from src.layer.flatten import Flatten
from src.layer.pooling import Pooling
from src.layer.convolutional import Convolutional

## Model

In [ ]:
model = 

## RNN Experiment

In [3]:
def perform_experiment(window_size=5):
    train_data_x, _ = main_rnn.get_data(window_size)

    model = Sequential()
    model.add(LSTM(input_shape=(window_size, 5), hidden_cell_dim=64, return_sequences=False))
    model.add(Dense(size=5, input_size=64, activation="linear"))
    predictions = model.run(inputs=train_data_x)

    ic(predictions)
    ic(predictions.shape)

    model.save_model(f"model-lstm-window-{window_size}.json")

### First Experiment

In [4]:
perform_experiment(4)

Finished processing data 500
Finished processing data 1000
Finished processing data 1500
Finished processing data 2000
Finished processing data 2500
Finished processing data 3000
Finished processing data 3500
Finished processing data 4000
Finished processing data 4500
Finished processing data 5000
Finished processing data 5500
Finished processing data 6000
Finished processing data 6500
Finished processing data 7000
Finished processing data 7500
Finished processing data 8000
Finished processing data 8500
Finished processing data 9000
Finished processing data 9500


ic| predictions: array([[28.96920544, 34.26420085, 34.70481304, 32.51309967, 33.83579421],
                        [28.96528403, 34.25914621, 34.69957444, 32.50820786, 33.83033819],
                        [28.96709362, 34.26148845, 34.70202923, 32.51046335, 33.83285617],
                        ...,
                        [28.97220823, 34.26845412, 34.70902572, 32.51716999, 33.84050345],
                        [28.96949891, 34.26498505, 34.70572653, 32.51386347, 33.83683859],
                        [28.9703461 , 34.26606287, 34.70676845, 32.51489668, 33.83799509]])
ic| predictions.shape: (9641, 5)


### Second Experiment

In [8]:
perform_experiment(10)

Finished processing data 500
Finished processing data 1000
Finished processing data 1500
Finished processing data 2000
Finished processing data 2500
Finished processing data 3000
Finished processing data 3500
Finished processing data 4000
Finished processing data 4500
Finished processing data 5000
Finished processing data 5500
Finished processing data 6000
Finished processing data 6500
Finished processing data 7000
Finished processing data 7500
Finished processing data 8000
Finished processing data 8500
Finished processing data 9000
Finished processing data 9500


ic| predictions: array([[34.89597599, 31.95423674, 30.92179856, 29.04766371, 34.03657876],
                        [34.89597595, 31.95423671, 30.92179852, 29.04766367, 34.03657872],
                        [34.89597597, 31.95423673, 30.92179854, 29.04766369, 34.03657874],
                        ...,
                        [34.89597601, 31.95423676, 30.92179857, 29.04766372, 34.03657877],
                        [34.89597601, 31.95423676, 30.92179857, 29.04766372, 34.03657877],
                        [34.89597599, 31.95423675, 30.92179856, 29.04766371, 34.03657876]])
ic| predictions.shape: (9635, 5)


### Third Experiment

In [6]:
perform_experiment(20)

Finished processing data 500
Finished processing data 1000
Finished processing data 1500
Finished processing data 2000
Finished processing data 2500
Finished processing data 3000
Finished processing data 3500
Finished processing data 4000
Finished processing data 4500
Finished processing data 5000
Finished processing data 5500
Finished processing data 6000
Finished processing data 6500
Finished processing data 7000
Finished processing data 7500
Finished processing data 8000
Finished processing data 8500
Finished processing data 9000
Finished processing data 9500


ic| predictions: array([[31.48821396, 31.48129857, 30.65705079, 31.11956083, 30.20502983],
                        [31.48821396, 31.48129857, 30.65705079, 31.11956083, 30.20502983],
                        [31.48821396, 31.48129857, 30.65705079, 31.11956083, 30.20502983],
                        ...,
                        [31.48821396, 31.48129857, 30.65705079, 31.11956083, 30.20502983],
                        [31.48821396, 31.48129857, 30.65705079, 31.11956083, 30.20502983],
                        [31.48821396, 31.48129857, 30.65705079, 31.11956083, 30.20502983]])
ic| predictions.shape: (9625, 5)


## CNN Experiment

In [7]:
train_data = main_cnn.get_data()
train_x, train_y = train_data["x"], train_data["y"]
train_x = np.asarray(train_x)

train_x = train_x.reshape((500, 3, 256, 256))

train_x = train_x.astype("float")
train_x /= 255

model = Sequential()
model.add(
    Convolutional(
        input_shape=(3, 256, 256),
        padding=0,
        filter_count=2,
        kernel_shape=(3, 3),
        stride=1,
    )
)
model.add(Detector(activation="relu"))
model.add(Pooling(size=(2, 2), stride=2, mode="max"))
model.add(Pooling(size=(5, 5), stride=5, mode="avg"))
model.add(Flatten())
model.add(Dense(size=10, input_size=1250, activation="relu"))
model.add(Dense(size=1, input_size=10, activation="sigmoid"))

predictions = model.run(inputs=train_x)

ic(predictions)

ic| predictions: array([[0.93199036],
                        [0.93199036],
                        [0.93199036],
                        [0.93199036],
                        [0.93199036],
                        [0.93199036],
                        [0.93199036],
                        [0.93199036],
                        [0.93199036],
                        [0.93199036],

Finished processing data 500



                        [0.93199036],
                        [0.93199036],
                        [0.93199036],
                        [0.93199036],
                        [0.93199036],
                        [0.93199036],
                        [0.93199036],
                        [0.93199036],
                        [0.93199036],
                        [0.93199036],
                        [0.93199036],
                        [0.93199036],
                        [0.93199036],
                        [0.93199036],
                        [0.93199036],
                        [0.93199036],
                        [0.93199036],
                        [0.93199036],
                        [0.93199036],
                        [0.93199036],
                        [0.93199036],
                        [0.93199036],
                        [0.93199036],
                        [0.93199036],
                        [0.93199036],
                        [0.93199036],
           

array([[0.93199036],
       [0.93199036],
       [0.93199036],
       [0.93199036],
       [0.93199036],
       [0.93199036],
       [0.93199036],
       [0.93199036],
       [0.93199036],
       [0.93199036],
       [0.93199036],
       [0.93199036],
       [0.93199036],
       [0.93199036],
       [0.93199036],
       [0.93199036],
       [0.93199036],
       [0.93199036],
       [0.93199036],
       [0.93199036],
       [0.93199036],
       [0.93199036],
       [0.93199036],
       [0.93199036],
       [0.93199036],
       [0.93199036],
       [0.93199036],
       [0.93199036],
       [0.93199036],
       [0.93199036],
       [0.93199036],
       [0.93199036],
       [0.93199036],
       [0.93199036],
       [0.93199036],
       [0.93199036],
       [0.93199036],
       [0.93199036],
       [0.93199036],
       [0.93199036],
       [0.93199036],
       [0.93199036],
       [0.93199036],
       [0.93199036],
       [0.93199036],
       [0.93199036],
       [0.93199036],
       [0.931